In [1]:

!pip install tensorboardx
!pip install transformers
!pip install simpletransformers


     |████████████████████████████████| 204kB 6.6MB/s 
     |████████████████████████████████| 645kB 6.5MB/s 
     |████████████████████████████████| 3.8MB 33.0MB/s 
     |████████████████████████████████| 1.1MB 34.2MB/s 
     |████████████████████████████████| 890kB 39.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=818e3342471260ee856bf9f3dba73fab4a81ecdbc734932ba28af5f24fb26914
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 6.1MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=1ba8f663a5a029dab6bb88854c30089332950c9f77273e496fbe8b9911f54343
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [2]:
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir  ./apex



Cloning into 'apex'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 6750 (delta 0), reused 1 (delta 0), pack-reused 6742
Receiving objects: 100% (6750/6750), 13.74 MiB | 13.37 MiB/s, done.
Resolving deltas: 100% (4505/4505), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-qmgpxbbn
Created temporary directory: /tmp/pip-req-tracker-8sepb32r
Created requirements tracker '/tmp/pip-req-tracker-8sepb32r'
Created temporary directory: /tmp/pip-install-ktpu0vaq
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-6_28qfcu
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-8sepb32r'
    Running setup.py (path:/tmp/pip-req-build-6_28qfcu/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-6_28qfcu/pip-egg-info/apex.e

In [3]:
import pandas as pd
import numpy as np
from transformers import *

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
stop = stopwords.words('english')
stop.pop(118)

from sklearn.model_selection import train_test_split

from simpletransformers.classification import ClassificationModel

from sklearn.metrics import mean_absolute_error, accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks")
os.listdir()


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['cuda_10.0.130_mac',
 'CS182 Final Project',
 'Untitled0.ipynb',
 'cache_dir',
 'outputs',
 'runs',
 'apex',
 'setup.sh',
 'outputs_roberta',
 'outputs_xlm',
 'outputs_xlnet']

In [0]:
review_path = "CS182 Final Project/data/yelp_review_training_dataset.jsonl"
data = pd.read_json(review_path, lines = True)


In [0]:
data = data.drop(["review_id"], axis =1)

In [8]:
data.head()

,text,stars
0,Total bill for this horrible service? Over $8G...,1
1,I *adore* Travis at the Hard Rock's new Kelly ...,5
2,I have to say that this office really has it t...,5
3,Went in for a lunch. Steak sandwich was delici...,5
4,Today was my second out of three sessions I ha...,1


In [9]:
data["text"] = data['text'].str.replace('[^\w\s]','').str.lower()
data["text"] = data['text'].str.replace('\d+', '')
data['text'] = data['text'].apply(lambda x: [item for item in x.split() if item not in stop])
data['text'] = data['text'].apply(lambda x: ' '.join(x))

data.head()

,text,stars
0,total bill horrible service gs crooks actually...,1
1,adore travis hard rocks new kelly cardenas sal...,5
2,say office really together organized friendly ...,5
3,went lunch steak sandwich delicious caesar sal...,5
4,today second three sessions paid although firs...,1


In [10]:
data.sample(frac=1)
data_sample = data.iloc[:10000]
data_sample.columns = ['text', 'labels']
train, validation = train_test_split(data_sample, test_size=0.2)
data_sample

,text,labels
0,total bill horrible service gs crooks actually...,1
1,adore travis hard rocks new kelly cardenas sal...,5
2,say office really together organized friendly ...,5
3,went lunch steak sandwich delicious caesar sal...,5
4,today second three sessions paid although firs...,1
...,...,...
9995,creative paradise neighbors back yard idea car...,1
9996,rugged warehouse interesting little store came...,3
9997,cant pinpoint exactly separates jjs sandwich s...,4
9998,police station pizza always favorite growing p...,5


In [0]:
model = ClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=6, use_cuda=True, args = {'output_dir' : "outputs_distilbert/"})

In [15]:
model.train_model(train)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.787365

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.657024Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.166540

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 2.130200Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.839839


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [16]:
result, model_outputs, wrong_predictions = model.eval_model(validation, avg_star_err=mean_absolute_error, exact_match=accuracy_score)

In [17]:
print(result)

{'mcc': 0.5686177089413013, 'avg_star_err': 0.427, 'exact_match': 0.6975, 'eval_loss': 0.7781316734552384}


In [0]:
predictions, raw_outputs = model.predict(["this restaurant is not good"])

In [0]:
print(predictions[0])
print(raw_outputs)